In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpt
%matplotlib inline
import re
from pprint import pprint
import seaborn as sns
sns.set()
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

# Cleaning the the csv

In [2]:
resto_df = pd.read_csv('ON_rest_reviews.csv')


In [3]:
resto_df = resto_df.drop(labels='Unnamed: 0', axis=1)
resto_df = resto_df.drop(labels=['useful','funny','cool','user_id'], axis=1)
#resto_df

In [4]:
review_texts=[]
for t in resto_df.text:
    temp = t.replace("@","")
    temp = temp.replace("^","")
    temp = temp.replace("*","")
    temp = temp.replace("{","")
    temp = temp.replace("}","")
    temp = temp.replace("[","")
    temp = temp.replace("]","")
    temp = temp.replace("|","")
    temp = temp.replace("~","")
    temp = temp.replace("&amp;","")
    temp = temp.replace("\n","")
    temp = temp.replace("\t","")
    temp = temp.replace("\\x","")
    temp = temp.replace(u'\u200b', '')
    temp = temp.replace(u'\u200d', '')
    temp = temp.replace(u'\u201c', '')
    temp = temp.replace(u'\ud83e', '')
    temp = temp.replace(u'\udd14', '')
    temp = temp.replace(u'\u2019', '')
    temp = temp.replace("\a","")
    temp = re.sub('\"+','',temp)
    temp = re.sub('\\+','',temp)
    temp = re.sub('http[s]?://\S+', '', temp)
    temp = re.sub(r'[^\x00-\x7F]+', '', temp)
    review_texts.append(temp)
    
resto_df['text'] = review_texts
#review_texts

In [5]:
resto_df['review_length'] = resto_df.text.map(len)
# check how the review lengths are distributed
#ax = sns.FacetGrid(data=resto_df, col='stars', xlim=(0, 2000)).map(mpt.hist, 'review_length', bins=50)
#ax.axes[0][0].set(ylabel='number of reviews');

In [6]:
positive_rev = resto_df.text[resto_df.stars>3].values
average_rev = resto_df.text[resto_df.stars==3].values
negative_rev = resto_df.text[resto_df.stars<3].values
#print('Postive Reviews:  {:,}'.format(len(pos_reviews)))
#print('Negative Reviews:  {:,}'.format(len(neg_reviews)))

# Removing stop words and Performing TF-IDF: Evaluation-1

In [7]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 


Wall time: 0 ns


### n-Gram Approach

In [8]:
stp_wrd_list = text.ENGLISH_STOP_WORDS.union(['know', 'like','best', 've','hype' ,'got', 'just', 
               'don', 'really', 'think', 'll' , 'returning' ,'said', 'told', 'ok', 'understand' ,'great', 'poor', 'worst','pad','thai',
                'khao', 'soi','came', 'make', 'sure', 'bad', 'dim', 'sum', 'love', 'place', 'highly', 'recommend', 'long',
                'okay' , 'definitely', 'come', 'extremely', 'customer' , 'slow', 'minutes', 'later', 'quality', 'terrible',
                'delicious', 'amazing', 'food', 'second', 'time', 'way', 'better', 'little', 'bit', 'excellent', 'years',
                'friend', 'ordered', 'wasn', 'busy','horrible', 'write', 'home', 'fast', 'slow', 'mi', 'friendly',
                'ago', 'right', 'away', 'worth', 'minute', 'minute wait', 'super', 'hit', 'miss', 'small', 'portions',
                'multiple', 'times', 'high', 'expectations', 'expectation', 'mins',
                'went', 'did', 'didn', 'good', 'pretty', ])

pos_tfidf  = TfidfVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')
avg_tfidf  = TfidfVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')
neg_tfidf  = TfidfVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')
np.random.seed(50)
selected_positive = np.random.choice(positive_rev,size=3000)
selected_average = np.random.choice(average_rev,size=3000)
selected_negative = np.random.choice(negative_rev,size=3000)
#print(selected_average)

In [9]:
replace_list = {'absolute': 'absolutely', 'accomodate':'accommodating', 'adds': 'add', 'adding': 'add', 'agree': 'agreed',
               'airy' : 'air', 'alcoholic':'alcohol', 'appetizers':'appetizer', 'bathrooms': 'bathroom', 'beans':'bean',
                'beers': 'beer', 'bringing': 'bring', 'burgers': 'burger', 'burrito': 'burritos', 'cakes': 'cake',
                'caramelized': 'caramel', 'ceilings': 'ceiling', 'cheesecakes':'cheesecake', 'chilli': 'chilly',
                'choices': 'choice', 'cocktails': 'cocktail', 'cookies': 'cookie', 'creaminess':'creamy',
                'crowds': 'crowded', 'desserts': 'dessert', 'drinks': 'drink', 'dumplings':'dumpling',
                'eggs':'egg', 'flavors': 'flavor', 'fruits': 'fruit', 'games': 'game', 'grilled': 'grill',
                'pizzas': 'pizza', 'tortillas': 'tortilla'}

### Replacing common words as similar bsed on above list

In [10]:
def clean_selected(t_list):
    cleaned = []
    for t in t_list:
        for i,j in replace_list.items():
            t = t.replace(i,j)
        cleaned.append(t)
    return cleaned
selected_positive = clean_selected(selected_positive)
selected_average = clean_selected(selected_average)
selected_negataive = clean_selected(selected_negative)

In [11]:
negative_vect = neg_tfidf.fit_transform(selected_negative)
average_vect = avg_tfidf.fit_transform(selected_average)
positive_vect = pos_tfidf.fit_transform(selected_positive)

D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [12]:
negative_df = pd.DataFrame(negative_vect.todense(), columns=[neg_tfidf.get_feature_names()])
average_df = pd.DataFrame(average_vect.todense(), columns=[avg_tfidf.get_feature_names()])
positive_df = pd.DataFrame(positive_vect.todense(), columns=[pos_tfidf.get_feature_names()])

In [13]:
#positive_df

In [14]:
mean_negative = {}
mean_average = {}
mean_positive = {}

for col_index in negative_df:
    mean_negative[col_index] = negative_df[col_index].mean()

for col_index in average_df:
    mean_average[col_index] = average_df[col_index].mean()
    
for col_index in positive_df:
    mean_positive[col_index] = positive_df[col_index].mean()

In [15]:
#mean_negative

In [16]:
print('Negative')
print(sorted(mean_negative, key=mean_negative.get, reverse=True)[:10])

print('Avg')
print(sorted(mean_average, key=mean_average.get, reverse=True)[:10])

print('Positive')
print(sorted(mean_positive, key=mean_positive.get, reverse=True)[:10])

Negative
[('ice cream',), ('wait staff',), ('fried chicken',), ('butter chicken',), ('deep fried',), ('overall experience',), ('save money',), ('mac cheese',), ('dining experience',), ('pulled pork',)]
Avg
[('ice cream',), ('pork belly',), ('fried chicken',), ('green tea',), ('deep fried',), ('mac cheese',), ('portion size',), ('overall experience',), ('friday night',), ('late night',)]
Positive
[('ice cream',), ('pork belly',), ('fried chicken',), ('deep fried',), ('pulled pork',), ('green tea',), ('service quick',), ('fish tacos',), ('fried rice',), ('chicken waffles',)]


In [17]:
### It can be seen that in all 3 categories of TF-IDF, the majority phrases are related to service and order. 
### Hence only using TF-IDF, not much business insights can be obtained and topic modelling is required.
### We will use the vectors generated by TF-IDF for topic modelling

In [18]:
del negative_df
del positive_df
del average_df
del resto_df

# LSA - Topic Modelling

In [19]:
#!pip install gensim

In [256]:
#For genism we need terms by docs, so taking transpose of 3 matrices

In [41]:
positive_corpus = matutils.Sparse2Corpus(positive_vect.transpose())
average_corpus = matutils.Sparse2Corpus(average_vect.transpose())
negative_corpus = matutils.Sparse2Corpus(negative_vect.transpose())

In [42]:
#Interchanging key, value pairs

In [43]:
# Row indices
positive_id2 = dict((v,k) for k,v in pos_tfidf.vocabulary_.items())
average_id2 = dict((v,k) for k,v in pos_tfidf.vocabulary_.items())
negative_id2 = dict((v,k) for k,v in pos_tfidf.vocabulary_.items())

In [44]:
positive_id2 = corpora.Dictionary.from_corpus(positive_corpus, id2word=positive_id2)
average_id2 = corpora.Dictionary.from_corpus(average_corpus, id2word=average_id2)
negative_id2 = corpora.Dictionary.from_corpus(negative_corpus, id2word=negative_id2)

In [46]:
#positive_topics_lsi = models.lsimulticore.LsiMulticore(positive_corpus, id2word=positive_id2, num_topics=5, passes=10, workers=15)

In [ ]:
average_topics_lsi = models.LsiModel(average_corpus, id2word=average_id2, num_topics=5)

In [ ]:
#negative_topics_lsi = models.LsiModel(negative_corpus, id2word=negative_id2, num_topics=12)

# Count Vectorizer and LDA

In [21]:
positive_cnt_vect = CountVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')
average_cnt_vect = CountVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')
negative_cnt_vect = CountVectorizer(stop_words=stp_wrd_list, min_df=10, max_df=0.5,ngram_range=(2,2), token_pattern='[a-z][a-z]+')

np.random.seed(50)
cnt_selected_negative = np.random.choice(negative_rev, size=10000)
cnt_selected_positive = np.random.choice(positive_rev, size=10000)
cnt_selected_average = np.random.choice(average_rev, size=10000)

cnt_selected_negative = clean_selected(cnt_selected_negative)
cnt_selected_positive = clean_selected(cnt_selected_positive)
cnt_selected_average = clean_selected(cnt_selected_average)

cnt_negative_vect = negative_cnt_vect.fit_transform(cnt_selected_negative).transpose()
cnt_positive_vect = positive_cnt_vect.fit_transform(cnt_selected_positive).transpose()
cnt_average_vect = average_cnt_vect.fit_transform(cnt_selected_average).transpose()

D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wait'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [22]:
# Convert sparse matrix of counts to a gensim corpus
negative_corp = matutils.Sparse2Corpus(cnt_negative_vect)
average_corp = matutils.Sparse2Corpus(cnt_average_vect)
positive_corp = matutils.Sparse2Corpus(cnt_positive_vect)

id2_pos = dict((v, k) for k, v in positive_cnt_vect.vocabulary_.items())
id2_avg = dict((v, k) for k, v in average_cnt_vect.vocabulary_.items())
id2_neg = dict((v, k) for k, v in negative_cnt_vect.vocabulary_.items())

In [23]:
%%time
positive_lda = models.ldamulticore.LdaMulticore(corpus=positive_corp, num_topics=5,id2word=id2_pos, passes=10, workers=15)
average_lda = models.ldamulticore.LdaMulticore(corpus=average_corp, num_topics=5,id2word=id2_avg, passes=10, workers=15)
negative_lda = models.ldamulticore.LdaMulticore(corpus=negative_corp, num_topics=5,id2word=id2_neg, passes=10, workers=15)

Wall time: 3min 8s


In [24]:

print("Positive : ")
pprint(positive_lda.print_topics(num_words=5))

print("Average : ")
pprint(average_lda.print_topics(num_words=5))

print("Negative : ")
pprint(negative_lda.print_topics(num_words=5))

Positive : 
[(0,
  '0.014*"fried chicken" + 0.009*"hot sauce" + 0.009*"fish chips" + '
  '0.007*"baja fish" + 0.006*"chicken katsu"'),
 (1,
  '0.013*"pork belly" + 0.013*"pulled pork" + 0.010*"green curry" + '
  '0.008*"spring rolls" + 0.008*"sweet potato"'),
 (2,
  '0.013*"pork belly" + 0.010*"mac cheese" + 0.008*"fried chicken" + '
  '0.006*"bone marrow" + 0.006*"smoked meat"'),
 (3,
  '0.031*"ice cream" + 0.014*"green tea" + 0.011*"deep fried" + 0.008*"duck '
  'fat" + 0.008*"late night"'),
 (4,
  '0.011*"french toast" + 0.009*"chicken waffles" + 0.008*"iced tea" + '
  '0.008*"kimchi fries" + 0.007*"egg benny"')]
Average : 
[(0,
  '0.018*"pork belly" + 0.005*"ramen places" + 0.005*"san road" + 0.005*"duck '
  'fat" + 0.004*"soon tofu"'),
 (1,
  '0.007*"smoked meat" + 0.005*"uncle tetsu" + 0.005*"cheese cake" + '
  '0.005*"beef brisket" + 0.004*"quite nice"'),
 (2,
  '0.011*"deep fried" + 0.006*"ice cream" + 0.006*"late night" + 0.006*"fried '
  'chicken" + 0.005*"hot sauce"'),
 (3,


# NMF

In [19]:
%%time
positive_nmf = NMF(n_components=5)

# for NMF, we know that A =WH where w is basis vector nd h is co-efficient vector

positive_W_Mat = positive_nmf.fit_transform(positive_vect)
positive_H_Mat = positive_nmf.components_

average_nmf = NMF(n_components=5)
average_W_Mat = average_nmf.fit_transform(average_vect)
average_H_Mat = average_nmf.components_

negative_nmf = NMF(n_components=5)
negative_W_Mat = negative_nmf.fit_transform(negative_vect)
negative_H_Mat = negative_nmf.components_

Wall time: 349 ms


In [94]:
def display_topics(model, feature_names, num_topics, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        if topic_idx < num_topics:
            print("{:11}".format("Topic %d:" %(topic_idx)), end='')
            print(", ".join(['{:04.3f}*'.format(topic[i])+feature_names[i] \
                             for i in topic.argsort()[:-no_top_words-1:-1]]))
            
no_topics = 5
no_top_words = 5

print("Positive: -------------")
display_topics(positive_nmf, pos_tfidf.get_feature_names(), no_topics, no_top_words)

print("Average: -------------")
display_topics(average_nmf, avg_tfidf.get_feature_names(), no_topics, no_top_words)

print("Negative: -------------")
display_topics(negative_nmf, neg_tfidf.get_feature_names(), no_topics, no_top_words)

Positive: -------------
Topic 0:   2.370*ice cream, 0.776*green tea, 0.388*black sesame, 0.329*soft serve, 0.149*whipped cream
Topic 1:   2.386*pork belly, 0.339*kimchi fries, 0.233*belly banh, 0.201*banh boys, 0.112*sweet potato
Topic 2:   1.917*fried chicken, 0.553*chicken waffles, 0.129*wait line, 0.121*french toast, 0.106*beef brisket
Topic 3:   1.895*deep fried, 0.396*big fan, 0.252*fish tacos, 0.183*fried rice, 0.167*hot sauce
Topic 4:   1.874*pulled pork, 0.493*pork sandwich, 0.248*bbq sauce, 0.226*service quick, 0.218*mac cheese
Average: -------------
Topic 0:   2.496*ice cream, 0.237*soft serve, 0.111*serve ice, 0.108*waitress nice, 0.096*tea ice
Topic 1:   2.391*fried chicken, 0.182*chicken thigh, 0.155*deep fried, 0.118*sweet liking, 0.106*fried rice
Topic 2:   1.873*pork belly, 0.267*kimchi fries, 0.223*soup base, 0.181*duck confit, 0.150*ramen places
Topic 3:   1.530*mac cheese, 0.490*deep fried, 0.477*late night, 0.406*sweet potato, 0.273*fish tacos
Topic 4:   1.890*green

In [95]:
#Taking a random positive review and anlysing it
def rev_analysis(sel_revs, W_Mat):
    rand_num = np.random.randint(0, high=len(sel_revs))
    print('Review Number: ',rand_num, '-------------')
    print(sel_revs[rand_num])
    print("------------")
    top = {}
    for index, j in enumerate(W_Mat[rand_num]):
        top[index] = j

In [97]:
rev_analysis(selected_positive, positive_W_Mat)

Review Number:  456 -------------
4.5 StarsAfter hearing tons of people talking about Smash, I finally went to try!Upon entering, it was pretty empty since I came at the awkward 3pm which is right after the lunch rush. You are greeted by some friendly waitresses and left to gawk at the awesome interior of the place. It's classy and casual at the same time. I feel like you could walk in with jeans or a little black dress. It reminded me of moxies, but with a classier and more upscale design.Now onto the food....Since I came with my friend who was starving, we mostly shared a few dishes, mostly appetizer. Let me start with the truffle mac and cheese though. This is the bomb. I usually don't indulge in mac and cheese but...... this was so creamy and decadent. The truffle definitely upgraded it to another level. The pasta that they used was shaped like a fancy swirl. It was served on a skillet for aesthetic purposes (and to keep it hot). This was definitely a highlight, but not for the fai

In [100]:
rev_analysis(selected_average, average_W_Mat)
#len(average_W_Mat)
#negative_vect

Review Number:  413 -------------
Don't see what the big fuss is about Wilbur, my second time here, had the burritos the first time (per my husband's recommendation). I would had given that a 3-star as well. My friend really liked Wilbur and so we met here again and this time I got the Shrimp Fajitas, there was no pepper next to any menu items to indicate that it was spicy. I doubt I'm the only one who wish they would include that. Their pineapple salsa from the bar is my favourite. Their line moves pretty quickly for ordering. It's waiting for the food that could take about 15 minutes in the heart of lunch rush.
------------
Topic 3: 0.0161
Topic 4: 0.0136
Topic 1: 0.0037
Topic 2: 0.0018
Topic 0: 0.0000


In [106]:
rev_analysis(selected_negative, negative_W_Mat)

Review Number:  1299 -------------
Disappointed. How do they claim to be an Italian restaurant when their pasta is worse than East Side Mario's?Spaghetti in Canna a Mare (fresh clams and mussels, calamari, scallops and tiger shrimp, light tomato sauce, mildly spicy) for $22.95 - Let me start off by saying that this pasta does not come with cheese :( but that's not even the problem. The pasta itself is SEVERELY UNDERCOOKED. I might as well have eaten the pasta raw and dry. It was extremely stiff and tough to chew on, my jaw was aching after 2 bites. The seafood was also undercooked, the mussels weren't even opened yet. Overall, it was bland. Very bland. There was no flavour to it whatsoever. It's not even a light pasta, it just had no seasoning whatsoever and it really lacked a sauce. I just ate all the raw seafood and left the pasta because I'm pretty sure I wouldn't be able to digest it. Seafood pasta is my all-time favourite pasta dish and Terroni has ruined it for me.Tortelloni di R

# Let's perform modelling to all the reviews

In [107]:
%%time

positive_rev = clean_selected(positive_rev)
average_rev = clean_selected(average_rev)
negataive_rev = clean_selected(negative_rev)

positive_vect = pos_tfidf.fit_transform(positive_rev)
average_vect = avg_tfidf.fit_transform(average_rev)
negative_vect = neg_tfidf.fit_transform(negative_rev)

Wall time: 22.5 s


In [109]:
print("Positive: -------------")
display_topics(positive_nmf, pos_tfidf.get_feature_names(), no_topics, no_top_words)

print("Average: -------------")
display_topics(average_nmf, avg_tfidf.get_feature_names(), no_topics, no_top_words)

print("Negative: -------------")
display_topics(negative_nmf, neg_tfidf.get_feature_names(), no_topics, no_top_words)

Positive: -------------
Topic 0:   3.186*pork belly, 1.682*kimchi fries, 0.736*spice pork, 0.517*banh boys, 0.476*belly banh
Topic 1:   2.283*ice cream, 0.897*green tea, 0.510*soft serve, 0.462*black sesame, 0.295*tea ice
Topic 2:   2.231*fried chicken, 0.555*chicken waffles, 0.207*deep fried, 0.179*chicken bao, 0.141*chicken dinner
Topic 3:   0.875*green curry, 0.471*duck fat, 0.462*fish tacos, 0.457*french toast, 0.431*fish chips
Topic 4:   2.213*pulled pork, 0.621*pork sandwich, 0.308*beef brisket, 0.274*mac cheese, 0.271*bbq sauce
Average: -------------
Topic 0:   2.759*ice cream, 0.639*soft serve, 0.249*serve ice, 0.226*tea ice, 0.174*egg waffles
Topic 1:   2.575*pork belly, 0.650*kimchi fries, 0.220*ramen places, 0.218*spice pork, 0.217*pulled pork
Topic 2:   0.630*mac cheese, 0.604*fish tacos, 0.525*late night, 0.373*french toast, 0.321*deep fried
Topic 3:   2.169*fried chicken, 0.285*chicken waffles, 0.235*deep fried, 0.141*chicken dinner, 0.117*pulled pork
Topic 4:   2.118*gre

# Renaming topics, to be used later

In [194]:
pos_dict = {0:'pork belly/kimchi fries', 1:'ice cream/green tea', 2:'fried chicken/ chicken waffles', 
              3:'green curry/ duck fat/ fish tacos', 4:'pork sandwich/ beef brisket/mac cheese'}

avg_dict = {0:'ice cream/egg waffles', 1:'pork belly/kimchi fries/ ramen places', 2:'mac cheese/ fish tacos', 
              3:'fried chicken/ deep fried', 4:'green tea/ red bean/cheesecake'}

neg_dict = {0:'ice cream/ service', 1:'waiting/ staff', 2:'deep fried/fish chips/ spirng rolls', 
              3:'butter chicken/ palak paneer', 4:'fried chicken/ smoked meat'}

In [117]:
positive_rev_df = resto_df[['business_id', 'stars', 'text', 'review_length']][resto_df.stars>3].reset_index()
average_rev_df = resto_df[['business_id', 'stars', 'text', 'review_length']][resto_df.stars==3].reset_index()
negative_rev_df = resto_df[['business_id', 'stars', 'text', 'review_length']][resto_df.stars<3].reset_index()
positive_rev_df.drop(labels='index', axis=1, inplace=True)
average_rev_df.drop(labels='index', axis=1, inplace=True)
negative_rev_df.drop(labels='index', axis=1, inplace=True)

### Normalizing the weights so they make 1 and are more readable 

In [119]:
positive_rev_df = pd.concat([positive_rev_df, pd.DataFrame(normalize(positive_W_Mat, norm='l1'))], axis=1)
average_rev_df = pd.concat([average_rev_df, pd.DataFrame(normalize(average_W_Mat, norm='l1'))], axis=1)
negative_rev_df = pd.concat([negative_rev_df, pd.DataFrame(normalize(negative_W_Mat, norm='l1'))], axis=1)

In [157]:
grp_mean_pos = positive_rev_df.groupby('business_id').agg({'stars':'mean', 'text':'count','review_length':'mean',0:'mean', 
                                                1:'mean', 2:'mean', 3:'mean',4:'mean'}).reset_index()
grp_mean_avg = average_rev_df.groupby('business_id').agg({'stars':'mean', 'text':'count','review_length':'mean',0:'mean', 
                                                 1:'mean', 2:'mean', 3:'mean',4:'mean'}).reset_index()
grp_mean_neg = negative_rev_df.groupby('business_id').agg({'stars':'mean', 'text':'count','review_length':'mean',0:'mean', 
                                                 1:'mean', 2:'mean', 3:'mean',4:'mean'}).reset_index()

In [158]:
grp_mean_pos.drop(labels='text', axis=1, inplace=True)


In [159]:
grp_mean_avg.drop(labels='text', axis=1, inplace=True)
grp_mean_neg.drop(labels='text', axis=1, inplace=True)

In [160]:
grp_mean_avg

,business_id,stars,review_length,0,1,2,3,4
0,-av1lZI1JDY_RZN2eTMnWg,3.0,642.062222,0.019575,0.024264,0.878158,0.016790,0.007879
1,0J_NiF5Lb0bFM5v6ZY-uGw,3.0,647.870370,0.017626,0.022931,0.836254,0.061339,0.024813
2,0a2O150ytxrDjDzXNfRWkA,3.0,802.539474,0.044317,0.026676,0.603201,0.021654,0.238362
3,0v8icS8wOOgEDiHDCOQkZQ,3.0,532.789474,0.023698,0.039304,0.677592,0.066322,0.035189
4,2Q89PhkcyT9uZnQORmqMKg,3.0,658.157143,0.026352,0.034682,0.827557,0.054777,0.028061
...,...,...,...,...,...,...,...,...
166,yY3jNsrpCyKTqQuRuLV8gw,3.0,1044.584615,0.021610,0.022739,0.873639,0.053528,0.013100
167,yg_A_TpYkJjr1fef0J6QkQ,3.0,887.258824,0.057241,0.443021,0.334613,0.095143,0.069981
168,zA6gnF5aPBGoOm6uIbKt-A,3.0,678.637681,0.059825,0.026937,0.726528,0.028232,0.115000
169,zfQ855VX3SMA_54oVSN5Cw,3.0,588.117647,0.010099,0.009711,0.860868,0.028444,0.032054


# Appending the restaurant details

In [165]:
on_resto = pd.read_csv('Ontario_resto.csv')
on_resto.drop(labels='Unnamed: 0', axis=1, inplace=True)

In [166]:
on_resto

,business_id,name,address,city,state,postal_code,stars,review_count,attributes,categories
0,c35qzLN6ItDpVIoj2uQW4Q,Ravi Soups,322 Adelaide Street W,Toronto,ON,M5V 1R1,4.5,332,"{'Alcohol': ""u'beer_and_wine'"", 'BikeParking':...","Soup, Restaurants, Sandwiches"
1,uF86ZhygpBEGr3CudNemYA,O.Noir,620 Church St,Toronto,ON,M4Y 2G2,3.5,301,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Canadian (New), American (New), Restaurants"
2,XYIPXJ9parr9FtvvcGI1SA,GB Hand-pulled Noodles,66 Edward Street,Toronto,ON,M5G 1C9,4.0,421,"{'RestaurantsDelivery': 'False', 'HasTV': 'Fal...","Chinese, Restaurants, Noodles"
3,WKOUTdVJS58E178JjhwidQ,Pastel Creperies & Dessert House,5417 Yonge Street,North York,ON,M2N 5R6,4.0,316,"{'RestaurantsDelivery': 'False', 'GoodForMeal'...","Food, Restaurants, Coffee & Tea, Creperies, De..."
4,O-uIEuv7JLUHajkemx_sVw,Beerbistro,18 King Street E,Toronto,ON,M5C 1C4,4.0,596,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Bistros, Bars, Beer Bar, Pubs, Ni..."
...,...,...,...,...,...,...,...,...,...,...
166,A7waf6G3cvnLfAqKeLL8DA,Buca,604 King Street W,Toronto,ON,M5V 1M6,4.0,392,"{'RestaurantsAttire': ""u'dressy'"", 'Caters': '...","Restaurants, Italian"
167,O66Zy8Y13VBm72ZDhS4fIg,Sassafraz,100 Cumberland Street,Toronto,ON,M5R 1A6,3.5,325,"{'BikeParking': 'True', 'RestaurantsPriceRange...","American (New), Restaurants, Event Planning & ..."
168,XCxxPZ3Lu5mwmIo7IQRf1g,Osaka Sushi Japanese Korean Restaurant,5762 Highway 7 E,Markham,ON,L3P 1A8,4.0,323,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Korean, Sushi Bars, Restaurants, Japanese"
169,5aeOewSy4RiI8sLLWpeNGA,Pablo Cheesetart Canada,114 Dundas St W,Toronto,ON,M5G 1C3,3.0,308,"{'RestaurantsPriceRange2': '2', 'RestaurantsDe...","Desserts, Food, Restaurants, Cafes"


In [188]:
final_pos = grp_mean_pos.merge(on_resto[['business_id', 'name', 'city', 'stars', 'review_count', 'address', 'postal_code']], left_on='business_id',right_on='business_id')[['business_id', 'name', 'address', 'postal_code' , 'city', 'stars_y','review_count',0, 1, 2, 3, 4]]

final_avg = grp_mean_avg.merge(on_resto[['business_id', 'name', 'city', 'stars', 'review_count', 'address', 'postal_code']], left_on='business_id', right_on='business_id')[['business_id', 'name', 'address', 'postal_code', 'city', 'stars_y','review_count', 0, 1, 2, 3, 4]]

final_neg = grp_mean_neg.merge(on_resto[['business_id', 'name', 'city', 'stars', 'review_count', 'address', 'postal_code']], left_on='business_id', right_on='business_id')[['business_id', 'name', 'address', 'postal_code', 'city', 'stars_y','review_count', 0, 1, 2, 3, 4]]

In [189]:
final_pos

,business_id,name,address,postal_code,city,stars_y,review_count,0,1,2,3,4
0,-av1lZI1JDY_RZN2eTMnWg,Salad King Restaurant,340 Yonge Street,M5B 1R8,Toronto,3.5,876,0.021768,0.025935,0.018437,0.910842,0.009212
1,0J_NiF5Lb0bFM5v6ZY-uGw,Zet's Restaurant,6445 Airport Road,L4V 1Y4,Mississauga,4.0,333,0.027666,0.027748,0.052672,0.733037,0.111818
2,0a2O150ytxrDjDzXNfRWkA,Miku,105-10 Bay Street,M5J 2R8,Toronto,4.0,604,0.018702,0.343642,0.012161,0.606169,0.011342
3,0v8icS8wOOgEDiHDCOQkZQ,Chilli Chicken House,"4040 Creditview Road, Unit 25",L5C,Mississauga,4.0,320,0.014346,0.028474,0.065532,0.863240,0.013083
4,2Q89PhkcyT9uZnQORmqMKg,House of Gourmet,484 Dundas St W,M5T 1G9,Toronto,3.5,345,0.025629,0.059035,0.029460,0.836558,0.040546
...,...,...,...,...,...,...,...,...,...,...,...,...
166,yY3jNsrpCyKTqQuRuLV8gw,Cactus Club Cafe,77 Adelaide Street W,M5H 1P9,Toronto,4.0,393,0.016287,0.040831,0.035991,0.880841,0.019201
167,yg_A_TpYkJjr1fef0J6QkQ,Konjiki Ramen,5051 Yonge Street,M2N 5P2,Toronto,4.0,393,0.258226,0.132209,0.068830,0.524764,0.008854
168,zA6gnF5aPBGoOm6uIbKt-A,Yuzu No Hana,236 Adelaide Street W,M5H 1W7,Toronto,4.0,498,0.022650,0.235822,0.013813,0.710082,0.006333
169,zfQ855VX3SMA_54oVSN5Cw,Fresco's Fish & Chips,201 Augusta Avenue,M5T 2L4,Toronto,4.5,320,0.011593,0.021616,0.033088,0.883727,0.032494


In [195]:
%%time
# Renaming columns

grp_mean_pos.rename(columns= pos_dict, inplace=True)
grp_mean_avg.rename(columns= avg_dict, inplace=True)
grp_mean_neg.rename(columns= neg_dict, inplace=True)


final_pos.rename(columns= pos_dict, inplace=True)
final_avg.rename(columns= avg_dict, inplace=True)
final_neg.rename(columns= neg_dict, inplace=True)


grp_mean_pos.to_csv('modelled_positive.csv')
grp_mean_avg.to_csv('modelled_average.csv')
grp_mean_neg.to_csv('modelled_negative.csv')
final_pos.to_csv('modelled_positive_full.csv')
final_avg.to_csv('modelled_average_full.csv')
final_neg.to_csv('modelled_negative_full.csv')

Wall time: 90.8 ms
